In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive

/content/drive/My Drive


In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras import backend as K
import os
from time import gmtime, strftime

from tensorflow.keras.optimizers import Adam
import json
import cv2 as cv
from sklearn.model_selection import train_test_split

import numpy as np
import h5py

In [ ]:
def createModel(input_shape,num_classes):
    base_model = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    predictions = Dense(num_classes, activation='sigmoid')(x)

    return Model(inputs=base_model.input, outputs=predictions)

def F1_score(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())

    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())

    return 2*((precision * recall) / (precision + recall + K.epsilon()))

def define_metrics():
    '''Define the training metrics.'''
    return [
      BinaryAccuracy(name='accuracy'),
      F1_score,
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
    ]

def define_callbacks(ckpt_path, monitor='val_loss'):
    '''Define the training callbacks.'''
    return [
        ModelCheckpoint(ckpt_path, monitor=monitor, verbose=0, save_best_only=True, 
                        save_weights_only=False, mode='auto', save_freq='epoch'),
        ReduceLROnPlateau(monitor=monitor, factor=0.1,
                              patience=6, min_lr=1e-13),
        EarlyStopping(monitor=monitor, patience=10, restore_best_weights=True)
    ]

def save_model(model, model_dir, model_name):
    '''Save the model in model_dir.'''
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    model_filename = model_name + '_' + strftime("%Y_%m_%d_%H_%M", gmtime()) + '.h5'
    model.save(os.path.join(model_dir, model_filename))
    print('Model saved at {}'.format(model_dir))


In [ ]:
with h5py.File('./dataset/dataset_final.hdf5', 'r') as f:
    X_train = f['X_train'][:]
    y_train = f['y_train'][:]
    X_val = f['X_val'][:]
    y_val = f['y_val'][:]

In [ ]:
# Define the training metrics

CLASSES = ["Beard", "Moustache", "Glasses"]

input_shape = X_train.shape[1:]
num_classes = len(CLASSES)

ckpt_path = "ckpt-15"
lr = 1e-4
model_dir ="models"
base_model = "resnet50"
num_epochs= 15


In [ ]:
#Create model
model = createModel(input_shape,num_classes)
model.summary()

metrics = define_metrics()

# Define callbacks
callbacks = define_callbacks(ckpt_path)

# Define the optimizer
opt = Adam(learning_rate = lr)

# Compile the model using binary_crossentropy
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=metrics)


94781440/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 206, 206, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 100, 100, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

In [ ]:
!pip install keras_vggface

In [ ]:
!pip install Keras-Applications

     |████████████████████████████████| 50 kB 2.9 MB/s 


In [ ]:
### CNN DEFINITION

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model, save, Model
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras_vggface.vggface import VGGFace

CLASSES = ["Beard", "Moustache", "Glasses"]
num_classes = len(CLASSES)

input_shape = X_train.shape[1:]

base_model = VGGFace(model="resnet50", include_top=False, input_shape=input_shape)
#base_model = tf.keras.applications.resnet.ResNet50(include_top=False, weights="imagenet", input_shape=input_shape)

base_model.trainable = True

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

INITIAL_EPOCH = 0
num_epochs = 15

ModuleNotFoundError: ignored

In [ ]:
metrics = define_metrics()
model_dir ="models"
# Define callbacks
ckpt_path = "ckpt-vgg"
callbacks = define_callbacks(ckpt_path)

# Define the optimizer
lr = 1e-4
opt = Adam(learning_rate = lr)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=metrics)

In [ ]:
#Fit model
model.fit(X_train,y_train, epochs=num_epochs, validation_data=(X_val,y_val), callbacks=callbacks, verbose=1)

Epoch 1/15
404/404 [==============================] - ETA: 0s - loss: 0.1216 - accuracy: 0.9522 - F1_score: 0.8761 - precision: 0.8859 - recall: 0.8670 - auc: 0.9846INFO:tensorflow:Assets written to: ckpt-vgg/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


404/404 [==============================] - 300s 697ms/step - loss: 0.1216 - accuracy: 0.9522 - F1_score: 0.8761 - precision: 0.8859 - recall: 0.8670 - auc: 0.9846 - val_loss: 0.0669 - val_accuracy: 0.9766 - val_F1_score: 0.9385 - val_precision: 0.9405 - val_recall: 0.9375 - val_auc: 0.9935 - lr: 1.0000e-04
Epoch 2/15
404/404 [==============================] - 244s 605ms/step - loss: 0.0493 - accuracy: 0.9820 - F1_score: 0.9510 - precision: 0.9567 - recall: 0.9512 - auc: 0.9969 - val_loss: 0.0876 - val_accuracy: 0.9693 - val_F1_score: 0.9218 - val_precision: 0.9040 - val_recall: 0.9396 - val_auc: 0.9922 - lr: 1.0000e-04
Epoch 3/15
404/404 [==============================] - ETA: 0s - loss: 0.0423 - accuracy: 0.9847 - F1_score: 0.9581 - precision: 0.9625 - recall: 0.9593 - auc: 0.9974INFO:tensorflow:Assets written to: ckpt-vgg/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


404/404 [==============================] - 277s 685ms/step - loss: 0.0423 - accuracy: 0.9847 - F1_score: 0.9581 - precision: 0.9625 - recall: 0.9593 - auc: 0.9974 - val_loss: 0.0556 - val_accuracy: 0.9786 - val_F1_score: 0.9460 - val_precision: 0.9196 - val_recall: 0.9736 - val_auc: 0.9975 - lr: 1.0000e-04
Epoch 4/15
404/404 [==============================] - 245s 605ms/step - loss: 0.0291 - accuracy: 0.9902 - F1_score: 0.9727 - precision: 0.9755 - recall: 0.9744 - auc: 0.9982 - val_loss: 0.0667 - val_accuracy: 0.9777 - val_F1_score: 0.9408 - val_precision: 0.9591 - val_recall: 0.9229 - val_auc: 0.9919 - lr: 1.0000e-04
Epoch 5/15
404/404 [==============================] - 244s 603ms/step - loss: 0.0281 - accuracy: 0.9910 - F1_score: 0.9744 - precision: 0.9784 - recall: 0.9757 - auc: 0.9986 - val_loss: 0.4168 - val_accuracy: 0.9526 - val_F1_score: 0.8648 - val_precision: 0.9401 - val_recall: 0.8038 - val_auc: 0.9390 - lr: 1.0000e-04
Epoch 6/15
404/404 [==============================] - 

...

404/404 [==============================] - 278s 687ms/step - loss: 0.0207 - accuracy: 0.9931 - F1_score: 0.9797 - precision: 0.9833 - recall: 0.9815 - auc: 0.9991 - val_loss: 0.0365 - val_accuracy: 0.9871 - val_F1_score: 0.9670 - val_precision: 0.9691 - val_recall: 0.9633 - val_auc: 0.9967 - lr: 1.0000e-04
Epoch 7/15
404/404 [==============================] - 244s 603ms/step - loss: 0.0119 - accuracy: 0.9960 - F1_score: 0.9872 - precision: 0.9896 - recall: 0.9901 - auc: 0.9995 - val_loss: 0.0484 - val_accuracy: 0.9851 - val_F1_score: 0.9609 - val_precision: 0.9677 - val_recall: 0.9542 - val_auc: 0.9941 - lr: 1.0000e-04
Epoch 8/15
404/404 [==============================] - 245s 606ms/step - loss: 0.0155 - accuracy: 0.9949 - F1_score: 0.9838 - precision: 0.9872 - recall: 0.9868 - auc: 0.9995 - val_loss: 0.0747 - val_accuracy: 0.9794 - val_F1_score: 0.9467 - val_precision: 0.9409 - val_recall: 0.9526 - val_auc: 0.9919 - lr: 1.0000e-04
Epoch 9/15
404/404 [==============================] - 

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


404/404 [==============================] - 274s 679ms/step - loss: 0.0025 - accuracy: 0.9995 - F1_score: 0.9964 - precision: 0.9989 - recall: 0.9987 - auc: 0.9999 - val_loss: 0.0346 - val_accuracy: 0.9917 - val_F1_score: 0.9790 - val_precision: 0.9795 - val_recall: 0.9774 - val_auc: 0.9953 - lr: 1.0000e-05
Epoch 14/15
404/404 [==============================] - 244s 603ms/step - loss: 0.0011 - accuracy: 0.9999 - F1_score: 0.9996 - precision: 0.9995 - recall: 0.9999 - auc: 1.0000 - val_loss: 0.0349 - val_accuracy: 0.9919 - val_F1_score: 0.9792 - val_precision: 0.9805 - val_recall: 0.9774 - val_auc: 0.9949 - lr: 1.0000e-05
Epoch 15/15
404/404 [==============================] - ETA: 0s - loss: 7.2603e-04 - accuracy: 0.9999 - F1_score: 0.9971 - precision: 0.9995 - recall: 0.9999 - auc: 1.0000INFO:tensorflow:Assets written to: ckpt-vgg/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


404/404 [==============================] - 276s 685ms/step - loss: 7.2603e-04 - accuracy: 0.9999 - F1_score: 0.9971 - precision: 0.9995 - recall: 0.9999 - auc: 1.0000 - val_loss: 0.0338 - val_accuracy: 0.9929 - val_F1_score: 0.9818 - val_precision: 0.9827 - val_recall: 0.9801 - val_auc: 0.9953 - lr: 1.0000e-05


In [ ]:
# Save the model
model_filename = 'model.h5'
save_model(model, model_dir, base_model)


TypeError: ignored

In [ ]:
model_filename = 'vgg_model'
save_model(model, model_dir, model_filename)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Model saved at models
